# Importação de biblioteca

In [10]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Leitura de CSVs

In [2]:
diamonds = pd.read_csv('/Users/rebeca/Downloads/asset-v1_IRONHACK+DAFT+202007_SAO+type@asset+block@diamonds.csv')
rdiamonds = pd.read_csv('/Users/rebeca/Downloads/asset-v1_IRONHACK+DAPT+202007_SAO+type@asset+block@rick_diamonds.csv')

# Analise exploratoria

In [3]:
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
1,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
2,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
3,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
4,0.24,Very Good,J,VVS2,62.8,57.0,336,3.94,3.96,2.48


In [4]:
rdiamonds.head()

,carat,cut,color,clarity,depth,table,x,y,z
0,0.91,Very Good,H,SI2,63.4,57.0,6.10,6.18,3.89
1,0.38,Good,F,VS2,60.3,61.0,4.68,4.71,2.83
2,0.52,Ideal,H,VS2,61.8,56.0,5.19,5.16,3.20
3,1.10,Ideal,G,VS1,61.3,56.0,6.65,6.69,4.09
4,2.28,Ideal,G,SI2,61.6,57.0,8.44,8.37,5.18


# limpeza de dados

# previsão

In [12]:
#pricemean = diamonds['price'].mean()
#rdiamonds['price_predicted'] = pricemean
#Your root mean squared error is 3980.713882502374

X = diamonds[['carat']]
y = diamonds['price']
model = LinearRegression()
model.fit(X,y)
model.predict(rdiamonds[['carat']])
rdiamonds['price_predicted'] = model.predict(rdiamonds[['carat']])
#Your root mean squared error is 1605.15175707362 
#Sem limpeza

# salvar o csv

In [13]:
rdiamonds.to_csv('resposta.csv', index=False)